In [1]:
cd /Users/pardhu/Desktop/Yashu/projects/leadScoringDemo

/Users/pardhu/Desktop/Yashu/projects/leadScoringDemo


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2, mutual_info_classif
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [43]:
data = pd.read_csv('Lead Scoring.csv')

In [4]:
data.shape

(9240, 37)

#### Features Not Select from EDA

* Do Not Call
* What matters most to you in choosing a course
* Search
* Magazine
* Newspaper Article
* X Education Forums
* Newspaper
* Digital Advertisement
* Through Recommendations
* Receive More Updates About Our Courses
* Update me on Supply Chain Content
* Get updates on DM Content
* I agree to pay the amount through cheque

In [5]:
no_info_features = [
    'Do Not Call',
    'What matters most to you in choosing a course',
    'Search',
    'Magazine',
    'Newspaper Article',
    'X Education Forums',
    'Newspaper',
    'Digital Advertisement',
    'Through Recommendations',
    'Receive More Updates About Our Courses',
    'Update me on Supply Chain Content',
    'Get updates on DM Content',
    'I agree to pay the amount through cheque']

In [6]:
cols_select = [col for col in data.columns if col not in no_info_features]

In [7]:
cols_select

['Prospect ID',
 'Lead Number',
 'Lead Origin',
 'Lead Source',
 'Do Not Email',
 'Converted',
 'TotalVisits',
 'Total Time Spent on Website',
 'Page Views Per Visit',
 'Last Activity',
 'Country',
 'Specialization',
 'How did you hear about X Education',
 'What is your current occupation',
 'Tags',
 'Lead Quality',
 'Lead Profile',
 'City',
 'Asymmetrique Activity Index',
 'Asymmetrique Profile Index',
 'Asymmetrique Activity Score',
 'Asymmetrique Profile Score',
 'A free copy of Mastering The Interview',
 'Last Notable Activity']

In [9]:
data_fin = data[cols_select]

In [10]:
data_fin.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,Tags,Lead Quality,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Interested in other courses,Low in Relevance,Select,Select,02.Medium,02.Medium,15.0,15.0,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Ringing,NaN,Select,Select,02.Medium,02.Medium,15.0,15.0,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Will revert after reading the email,Might be,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Ringing,Not Sure,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Will revert after reading the email,Might be,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,Modified


In [11]:
data_fin.shape

(9240, 24)

### Before doing ML  - We need to do FE - Selection

* Do we need to select features or just go with all?
* What type of Features we have?
* All categorical ?
* All numerical ?
* Categorical and Numerical?

###### If all categorical, how can we select right features for machine learning model 

* Chi-Squared Test
* Mutual Information
* Feature Importance from Ensemble Models
* Recursive Feature Elimination (RFE) with Cross-Validation

###### If all numerical, how can we select right features for machine learning model 

* Multicollinearity Check
* PCA
* ANOVA

###### If features are numerical and categorical, how can we select right features for machine learning model 

* Will fix above methods


* We have to check Target Feature


#### Dealing Catogrical Columns

In [30]:
numerical_columns = ['TotalVisits', 'Total Time Spent on Website', 
                     'Page Views Per Visit', 'Asymmetrique Activity Score',
                     'Asymmetrique Profile Score','Converted']
target_column_id = ['Converted','Prospect ID','Lead Number']
cols_filter = numerical_columns + target_column_id

In [29]:
categorical_column = [col for col in data_fin.columns if col not in cols_filter]

### Chi-square test 

* A p-value < 0.05 (or other thresholds like 0.01 or 0.001, depending on the strictness of your criteria) typically means that the feature is significantly related to the target variable and should be considered for inclusion in the model.

* A p-value >= 0.05 indicates that the feature may not be significantly related to the target, and you might consider dropping it or further analyzing it.

In [22]:
le = LabelEncoder()
alpha = 0.05
chi2_results = {}

for column in categorical_column:
    data[column] = le.fit_transform(data[column])
    chi2_stat, p_val = chi2(data[[column]], data['Converted'])
    significance = 'Significant' if p_val[0] < alpha else 'Not Significant'
    chi2_results[column] = {
        'Chi-Square Statistic': chi2_stat[0], 
        'p-value': p_val[0], 
        'Significance': significance
    }
pd.DataFrame(chi2_results).T

,Chi-Square Statistic,p-value,Significance
Lead Origin,264.27843,0.0,Significant
Lead Source,724.299542,0.0,Significant
Do Not Email,156.357525,0.0,Significant
Last Activity,1027.046231,0.0,Significant
Country,204.837336,0.0,Significant
Specialization,777.074677,0.0,Significant
How did you hear about X Education,359.974662,0.0,Significant
What is your current occupation,96.67657,0.0,Significant
Tags,1.460646,0.226827,Not Significant
Lead Quality,2034.398143,0.0,Significant


#### Dealing Numerical Columns

In [27]:
data_fin[numerical_columns].corr()

,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
TotalVisits,1.000000,0.221240,0.512125,-0.058514,0.130887
Total Time Spent on Website,0.221240,1.000000,0.320361,-0.065287,0.175589
Page Views Per Visit,0.512125,0.320361,1.000000,-0.162892,0.161254
Asymmetrique Activity Score,-0.058514,-0.065287,-0.162892,1.000000,-0.123250
Asymmetrique Profile Score,0.130887,0.175589,0.161254,-0.123250,1.000000


No Significant multicolinearity between features

* other alternate method is VIF 

In [32]:
### using Stattics Methods - Mean
data_fin[numerical_columns].groupby('Converted').mean()

,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
Converted,,,,,
0,3.329670,330.404473,2.368453,14.123956,16.035003
1,3.633632,738.546757,2.353638,14.603774,16.850629


In [33]:
### using Stattics Methods - Median 
data_fin[numerical_columns].groupby('Converted').median()

,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
Converted,,,,,
0,3.0,179.0,2.0,14.0,15.0
1,3.0,832.0,2.0,15.0,17.0


In [39]:
from scipy.stats import pointbiserialr
r, p_val = pointbiserialr(data['Converted'], data['Total Time Spent on Website'])
print(f"Point Biserial Correlation: {r}, p-value: {p_val}")

Point Biserial Correlation: 0.36248285688958826, p-value: 6.0615322149442544e-285


In [ ]:
from scipy.stats import pointbiserialr
r, p_val = pointbiserialr(data['Converted'], data['TotalVisits'].fillna(0))
print(f"Point Biserial Correlation: {r}, p-value: {p_val}")

In [40]:
from scipy.stats import pointbiserialr
r, p_val = pointbiserialr(data['Converted'], data['Page Views Per Visit'].fillna(0))
print(f"Point Biserial Correlation: {r}, p-value: {p_val}")

Point Biserial Correlation: -0.014725343791258139, p-value: 0.15696364333321955


W.R.T target label converted - #Total Time Spent on Website is Significant feature

#### Final Features for Model Building

* Total Time Spent on Website
* Lead Origin
* Lead Source
* Do Not Email
* Last Activity
* Specialization
* How did you hear about X Education
* What is your current occupation
* Lead Quality
* Lead Profile
* City

In [46]:
features_data = data[[
    'Total Time Spent on Website',
    'Lead Origin',
    'Lead Source',
    'Do Not Email',
    'Last Activity',
    'Specialization',
    'How did you hear about X Education',
    'What is your current occupation',
    'Lead Quality',
    'Lead Profile',
    'City', 'Converted'
]]

In [47]:
features_data.head()

,Total Time Spent on Website,Lead Origin,Lead Source,Do Not Email,Last Activity,Specialization,How did you hear about X Education,What is your current occupation,Lead Quality,Lead Profile,City,Converted
0,0,API,Olark Chat,No,Page Visited on Website,Select,Select,Unemployed,Low in Relevance,Select,Select,0
1,674,API,Organic Search,No,Email Opened,Select,Select,Unemployed,NaN,Select,Select,0
2,1532,Landing Page Submission,Direct Traffic,No,Email Opened,Business Administration,Select,Student,Might be,Potential Lead,Mumbai,1
3,305,Landing Page Submission,Direct Traffic,No,Unreachable,Media and Advertising,Word Of Mouth,Unemployed,Not Sure,Select,Mumbai,0
4,1428,Landing Page Submission,Google,No,Converted to Lead,Select,Other,Unemployed,Might be,Select,Mumbai,1


In [48]:
features_data.shape

(9240, 12)

In [49]:
### Modifying 
categorical_columns = ['Lead Origin',
    'Lead Source',
    'Do Not Email',
    'Last Activity',
    'Specialization',
    'How did you hear about X Education',
    'What is your current occupation',
    'Lead Quality',
    'Lead Profile',
    'City']

In [50]:
### Applying Label Encoder 

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    features_data[col] = le.fit_transform(features_data[col])
    label_encoders[col] = le

/var/folders/yx/plt06p490v30f5w_f7hdkdt00000gn/T/ipykernel_63805/627635212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data[col] = le.fit_transform(features_data[col])
/var/folders/yx/plt06p490v30f5w_f7hdkdt00000gn/T/ipykernel_63805/627635212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data[col] = le.fit_transform(features_data[col])
/var/folders/yx/plt06p490v30f5w_f7hdkdt00000gn/T/ipykernel_63805/627635212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [52]:
import pickle

# Save label encoders
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [56]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_data['Total Time Spent on Website'] = scaler.fit_transform(features_data[['Total Time Spent on Website']])
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

/var/folders/yx/plt06p490v30f5w_f7hdkdt00000gn/T/ipykernel_63805/2278972877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data['Total Time Spent on Website'] = scaler.fit_transform(features_data[['Total Time Spent on Website']])


In [57]:
features_data.head()

,Total Time Spent on Website,Lead Origin,Lead Source,Do Not Email,Last Activity,Specialization,How did you hear about X Education,What is your current occupation,Lead Quality,Lead Profile,City,Converted
0,-0.889974,0,6,0,10,15,6,4,1,4,4,0
1,0.339972,0,7,0,5,15,6,4,5,4,4,0
2,1.905689,1,1,0,5,1,6,3,2,3,0,1
3,-0.333396,1,1,0,13,11,9,4,3,4,0,0
4,1.715905,1,3,0,1,15,4,4,2,4,0,1


In [58]:
features_data.to_csv('model_data.csv')

In [59]:
features_data.columns

Index(['Total Time Spent on Website', 'Lead Origin', 'Lead Source',
       'Do Not Email', 'Last Activity', 'Specialization',
       'How did you hear about X Education', 'What is your current occupation',
       'Lead Quality', 'Lead Profile', 'City', 'Converted'],
      dtype='object')